# Configuring and Using Dataset Features 

This notebook shows model developers how they can learn more about configuring and using the features in `init_dataset` These instructions include the code required to:

- ...
- ...

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::


This notebook requires an OpenAI API secret key to run. If you don't have one, visit [API keys](https://platform.openai.com/account/api-keys) on OpenAI's site to create a new key for yourself. Note that API usage charges may apply.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [1]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Foundation Model - Text Sentiment Analysis`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace this placeholder with the code snippet from your own project ##

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

  

2023-12-14 17:01:49,821 - INFO(validmind.api_client): Connected to ValidMind. Project: Customer Churn Demo (1.4) - Initial Validation-demo (clnup756d051w15lf2dmzywvf)


## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [3]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the sample dataset

In [4]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn as demo_dataset

# You can also try a different dataset with:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

In [5]:
demo_dataset.target_column

'Exited'

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
7995,800,France,Female,29,2,0.00,2,0,0,167773.55,0
7996,771,France,Male,39,5,0.00,2,1,0,96270.64,0
7997,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
7998,709,France,Female,36,7,0.00,1,0,1,42085.58,1


### Initialize the training and test datasets

Before you can run a test suite, which are just a collection of tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module. 

This function takes a number of arguments: 

- `dataset` — the raw dataset that you want to analyze
- `target_column` — the name of the target column in the dataset 
- `feature_columns` - the names of the feature columns in the dataset 

With the datasets ready, you can now initialize the training and test datasets (`train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [7]:
feature_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns
)

2023-12-14 17:01:50,040 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Defining custom features

This section shows how defining custom features enables us to:

- Use these custom features to compute dataset metrics.
- Employ the columns not included as features for additional purposes, such as segmenting the metrics.

In the following example we use the metric `DescriptiveStatistics` to show how the output changes when customizing features.




1. Running metric with all the features.

In [8]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column
)

test = vm.tests.run_test(
    test_id="validmind.data_validation.DescriptiveStatistics",
    dataset=vm_dataset,
)

2023-12-14 17:01:50,091 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


2. Running metric with a subset of features.

In [9]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
    feature_columns=['CreditScore', 'Age', 'Balance', 'Geography']
)

test = vm.tests.run_test(
    test_id="validmind.data_validation.DescriptiveStatistics",
    dataset=vm_dataset,
)

2023-12-14 17:01:50,177 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
